In [1]:
import numpy as np
from time import time
from uuid import uuid4
import sys
from pprint import pprint as pp
import dataset
import pickle
from subgraph import query_to_kg_embeddings
import vdblite_but_better as vdblite

from tqdm.contrib.concurrent import process_map
import multiprocessing as mp
import time

[nltk_data] Downloading package punkt to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
file = open("simple_wikidata5m.pkl", "rb")
model = pickle.load(file)
entity2id = model.graph.entity2id
relation2id = model.graph.relation2id
entity_embeddings = model.solver.entity_embeddings
relation_embeddings = model.solver.relation_embeddings
alias2entity = dataset.wikidata5m.load_alias("./entity.txt.gz")

In [3]:
entity_db = vdblite.Vdb()
id2entity = model.graph.id2entity

In [4]:
entity2alias = {v: k for k, v in alias2entity.items()}

In [5]:
print(type(entity2alias))
for i in entity2alias:
    print(i, entity2alias[i])
    break

<class 'dict'>
Q5196650 cut your hair


In [6]:
for i in range(len(entity_embeddings)):
    entity_db.add({'entity_embedding': entity_embeddings[i], 'entity_id': i})
entity_db.finalize()

In [7]:
entity_db.details()

DB elements #: 4818298
DB size in memory: 39064728 bytes


In [8]:
# Pass the output of query_to_kg_embeddings
# 
def get_neighbourhood(query_embeddings):
    neighbours = {}
    qe = np.stack(list(query_embeddings.values()), axis=0)
    nearest = entity_db.search_many(qe, field='entity_embedding', count=5).astype(int)
    # print("Nearest: ", len(nearest))
    set_of_nearest = set(nearest.reshape(-1))
    for idx in set_of_nearest:
        n = entity_db.data[idx]
        try:
            alias = entity2alias[id2entity[n["entity_id"]]]
            neighbours[alias] = id2entity[n["entity_id"]]
        except Exception as e:
            pass
    return neighbours

In [9]:
temp = entity_embeddings[entity2id[alias2entity["steve jobs"]]]
query_embedding = {"steve jobs": temp}
#query_embedding

In [10]:
neighbours = get_neighbourhood(query_embeddings=query_embedding)
for n in neighbours:
    print(n, neighbours[n])

steve jobs dies Q19837
chrisann brennan Q18810921
dr. ed catmull Q93161
laurene jobs Q3133593
eric emerson schmidt Q92747


In [11]:
import os
import tqdm
halueval_path = os.path.join(os.getcwd(), "submodules", "HaluEval", "data", "qa_data.json")
print(halueval_path)

/var/home/zaristei/repos/KG-LLM-Hallucination/submodules/HaluEval/data/qa_data.json


In [12]:
import json
with(open(halueval_path, "r")) as f:
    data_lst = f.readlines()

data_lst = [json.loads(data) for data in data_lst]

In [13]:
RELEVANT_KEYS = ['knowledge', 'question']

In [14]:
def process_data(data):
    result = {}
    for k in RELEVANT_KEYS:
        query_embedding = query_to_kg_embeddings(data[k])
        #neighbours = get_neighbourhood(query_embedding)
        #result.update(neighbours)
        result.update(query_embedding)
    return result

In [15]:
processed_data = [process_data(i) for i in data_lst[:2]]

In [16]:
print(len(processed_data))
print(data_lst[0], processed_data[0])

2
{'knowledge': "Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.First for Women is a woman's magazine published by Bauer Media Group in the USA.", 'question': "Which magazine was started first Arthur's Magazine or First for Women?", 'right_answer': "Arthur's Magazine", 'hallucinated_answer': 'First for Women was started first.'} {"arthur's magazine": array([-5.64293526e-02, -1.44366249e-01,  1.32993281e-01,  7.50671849e-02,
       -3.08150023e-01, -1.74185142e-01, -2.75961965e-01, -2.04036877e-01,
        2.53782809e-01,  2.13799775e-02,  1.74351946e-01, -1.79804370e-01,
       -1.55779392e-01,  3.69694009e-02,  3.90264727e-02, -1.57269776e-01,
        1.30533040e-01, -1.82429790e-01, -1.82213888e-01, -1.27985328e-01,
       -3.34116459e-01, -2.14496598e-01, -1.21826328e-01,  1.41624227e-01,
       -3.81392501e-02,  2.99491286e-02, -2.54535884e-01, -1.01266704e-01,
        1.24744974e-01,  9.51925218e-02,  1.92483336e-01,

In [17]:
mp.cpu_count()
start = time.time()
result = process_map(process_data, data_lst, max_workers=16)
'''
for data in tqdm.tqdm(data_lst):
    result.append(process_data(data))
'''
end = time.time()
print(end-start)

/tmp/ipykernel_41191/1342579337.py:3: TqdmWarning: Iterable length 10000 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  result = process_map(process_data, data_lst, max_workers=16)


  0%|          | 0/10000 [00:00<?, ?it/s]

520.0145592689514


In [23]:
start = time.time()
neighbours = []
for data in tqdm.tqdm(result):
    neighbours.append(get_neighbourhood(data))
print(time.time()-start)

100%|██████████| 10000/10000 [05:00<00:00, 33.33it/s]

300.075909614563


In [25]:
import pickle as pkl
pkl.dump(neighbours, open("neighbours.pkl", "wb"))